In [1]:
import sys
import os
import time
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
data_df = pd.read_csv('/kaggle/input/emotion-classification/emotion.data')
data_df.columns = ['id', 'text', 'emotion']

In [3]:
def load_embeddings(file):
    """
    input: embeddings file
    output: embedding index
    """
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
    return embeddings_index

In [4]:
emb_folder = "/kaggle/input/glove6b50dtxt/"
emb_file_name = 'glove.6B.50d.txt'
emb_path = os.path.join(emb_folder, emb_file_name)
emb_glove = load_embeddings(emb_path)

In [5]:
data_df.head()

,id,text,emotion
0,27383,i feel awful about it too because it s my job ...,sadness
1,110083,im alone i feel awful,sadness
2,140764,ive probably mentioned this before but i reall...,joy
3,100071,i was feeling a little low few days back,sadness
4,2837,i beleive that i am much more sensitive to oth...,love


In [6]:
def build_vocabulary(texts):
    """
    input: list of list of words
    output: dictionary of words and their count
    """
    print('build vocabulary')
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in tqdm(sentences):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

import operator
def check_coverage(vocab, embeddings_index):
    '''
    input: vocabulary, embedding index
    output: list of unknown words; also prints the vocabulary coverage of embeddings and the % of comments text covered by the embeddings
    '''
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in tqdm(vocab.keys()):
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass
    print('Found embeddings for {:.3%} of vocabulary'.format(len(known_words)/len(vocab)))
    print('Found embeddings for {:.3%} of all text'.format(nb_known_words/(nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]
    return unknown_words                        


In [7]:
vocabulary = build_vocabulary(data_df['text'])
print(f'Vocabulary size: {len(vocabulary)}')

build vocabulary


100%|██████████| 416809/416809 [00:02<00:00, 169725.69it/s]


Vocabulary size: 75302


In [8]:
oof_emb = check_coverage(vocabulary, emb_glove)

100%|██████████| 75302/75302 [00:00<00:00, 573383.10it/s]

Found embeddings for 73.889% of vocabulary
Found embeddings for 99.549% of all text


In [9]:
oof_emb[0:20]

[('href', 4788),
 ('shouldnt', 777),
 ('hadnt', 394),
 ('youve', 337),
 ('werent', 269),
 ('itll', 220),
 ('nofollow', 202),
 ('hadn', 198),
 ('theyve', 183),
 ('everyones', 158),
 ('theyll', 152),
 ('pagetitle', 122),
 ('hahaha', 114),
 ('wouldve', 95),
 ('permalink', 92),
 ('theyd', 87),
 ('idk', 72),
 ('anyones', 70),
 ('everythings', 67),
 ('feedlinks', 62)]

In [10]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}


In [11]:
def known_contractions(embed):
    '''
    input: embedding matrix
    output: known contractions (from embeddings)
    '''
    known = []
    for contract in tqdm_notebook(contraction_mapping):
        if contract in embed:
            known.append(contract)
    return known

def clean_contractions(text, mapping=contraction_mapping):
    '''
    input: current text, contraction mappings
    output: modify the comments to use the base form from contraction mapping
    '''
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

In [12]:
data_df['text'] = data_df['text'].apply(lambda x: clean_contractions(x))

In [13]:
vocabulary = build_vocabulary(data_df['text'])
print(f'Vocabulary size: {len(vocabulary)}')

build vocabulary


100%|██████████| 416809/416809 [00:02<00:00, 177681.62it/s]


Vocabulary size: 75291


In [14]:
oof_emb = check_coverage(vocabulary, emb_glove)

100%|██████████| 75291/75291 [00:00<00:00, 561259.72it/s]

Found embeddings for 73.888% of vocabulary
Found embeddings for 99.549% of all text


In [15]:
new_contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization',
                      "href": " ", "shouldnt": "should not", "hadnt": "had not", "werent": "were not", "itll": "it will",
                      "nofollow": "no follow", "theyve":"they have", "everyones": "everyone", "theyll": "they will"}


In [16]:
#apply new entried contractions clean
data_df['text'] = data_df['text'].apply(lambda x: clean_contractions(x,new_contraction_mapping))

# build the vocabulary
vocabulary = build_vocabulary(data_df['text'])
print(f'Vocabulary size: {len(vocabulary)}')

# check the vocabulary coverage
oof_emb = check_coverage(vocabulary, emb_glove)

build vocabulary


100%|██████████| 75282/75282 [00:00<00:00, 590028.63it/s]

Vocabulary size: 75282
Found embeddings for 73.897% of vocabulary
Found embeddings for 99.638% of all text


In [17]:
print(f'Not covered vocabulary: {len(oof_emb)}; first 20 most frequent: {list(oof_emb[0:20])}')

Not covered vocabulary: 19651; first 20 most frequent: [('youve', 337), ('hadn', 198), ('pagetitle', 122), ('hahaha', 114), ('wouldve', 95), ('permalink', 92), ('theyd', 87), ('idk', 72), ('anyones', 70), ('everythings', 67), ('feedlinks', 62), ('languagedirection', 62), ('isprivate', 62), ('utf', 59), ('couldve', 57), ('pissy', 55), ('mobileclass', 51), ('ismobile', 51), ('shouldve', 47), ('onclick', 45)]


In [18]:
import numpy as np
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Input, LSTM, CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, SpatialDropout1D, Embedding, add
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import pickle

Using TensorFlow backend.


In [19]:
# Configurations
EMBED_SIZE = 50  # size of word vector; this should be set to 50 to match the embedding source
MAX_FEATURES = 200000  # how many unique words to use (i.e num rows in embedding vector)
MAXLEN = 50 # max length of  text

NO_EPOCHS = 30  # training epochs
VERBOSE = 1  # report frequency while training
PATIENCE = 5  # number of training epochs for validation not improved before stoping training
BATCH_SIZE = 128  # size of training batches
LSTM_UNITS = 128  # size of LSTM units
DENSE_HIDDEN_UNITS = 256  # size of dense hidden units

In [20]:
def tokenize(texts):
    tokenizer = Tokenizer(num_words=MAX_FEATURES)
    tokenizer.fit_on_texts(texts)
    word_index = tokenizer.word_index
    print(f"Found {len(word_index)} unique tokens.")
    return word_index, tokenizer

In [21]:
def build_embedding_matrix(word_index, emb_glove):
    print('build embedding matrix')
    embeddings_index = emb_glove
    embedding_matrix = np.zeros((len(word_index) + 1, EMBED_SIZE))
    for word, i in tqdm(word_index.items()):
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [22]:
def prepare_model(data_df, tokenizer):
    print("build model")

    print("label encoding")

    sequences = tokenizer.texts_to_sequences(data_df['text_proc'])
    data = pad_sequences(sequences, maxlen=MAXLEN)
    labels = data_df.emotion.values
    labels = to_categorical(np.asarray(labels))

    X = data
    y = labels

    x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

    print('Shape of data tensor:', X.shape)
    print('Shape of label tensor:', y.shape)
    print(f'Data train/valid: {x_train.shape}/{x_val.shape} Label train/valid:  {y_train.shape}/{y_val.shape}')
    return x_train, x_val, y_train, y_val


In [23]:
def build_train_model(word_index, embedding_matrix, x_train, x_val, y_train, y_val):
    DENSE_OUTPUT = y_val.shape[1]
    earlystopper = EarlyStopping(monitor='val_loss', patience=PATIENCE, verbose=VERBOSE)
    checkpointer = ModelCheckpoint('best_model.b5',
                                    monitor='val_acc',
                                    verbose=VERBOSE,
                                    save_best_only=True,
                                    save_weights_only=True)

    embedding_layer = Embedding(len(word_index) + 1,
                                EMBED_SIZE,
                                weights=[embedding_matrix],
                                input_length=MAXLEN,
                                trainable=False)

    model = None    
    sequence_input = Input(shape=(MAXLEN,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    x = SpatialDropout1D(0.4)(embedded_sequences)
    x = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(x)
    hidden = GlobalMaxPooling1D()(x)
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    result = Dense(DENSE_OUTPUT, activation='sigmoid')(hidden)

    model = Model(sequence_input, result)
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['acc'])
    print("run model - run and save the model pipeline")
    train_model  = model.fit(x_train, y_train, validation_data=[x_val, y_val],
                        epochs=NO_EPOCHS,
                        batch_size=BATCH_SIZE,
                        callbacks=[earlystopper, checkpointer],verbose=VERBOSE)
    print("run model - predict validation set")
    return model


In [24]:
def validation_model(model, x_val, y_val):
    print("run model - predict validation set")
    score = model.evaluate(x_val, y_val, verbose=0)
    print(f'Last validation loss: {score[0]}, accuracy: {score[1]}')
    # load saved optimal model
    model_optimal = model
    model_optimal.load_weights('best_model.b5')
    score = model_optimal.evaluate(x_val, y_val, verbose=0)
    print(f'Best validation loss: {score[0]}, accuracy: {score[1]}')

    print("run model - check prediction accuracy | precision | recall | F1-score")
    y_pred = model_optimal.predict(x_val)
    true_val = np.argmax(y_val, axis=1)
    pred_val = np.argmax(y_pred, axis=1)
    print(classification_report(true_val, pred_val))
    print("run model - completed validation")
    pickle.dump(model_optimal,open(os.path.join('.', "emotion.model"),'wb'))
    print("best model saved")

In [25]:
def build_run_model(data_df, embeddings):
    word_index, tokenizer = tokenize(data_df['text_proc'])
    embedding_matrix = build_embedding_matrix(word_index, embeddings)
    x_train, x_val, y_train, y_val = prepare_model(data_df, tokenizer)
    model = build_train_model(word_index, embedding_matrix, x_train, x_val, y_train, y_val)
    validation_model(model, x_val, y_val)

In [26]:
#SAMPLE = 50000
from sklearn.preprocessing import LabelEncoder
#train = data_df.sample(SAMPLE)
train = data_df
train['text_proc'] = train['text']
train_y = train.emotion.values
le = LabelEncoder()
le.fit(train_y)
train.emotion = le.transform(train_y)
le_name_mapping = dict(zip(le.transform(le.classes_), le.classes_))

In [27]:
import warnings
warnings.filterwarnings("ignore")

In [28]:
word_index, tokenizer = tokenize(data_df['text_proc'])
embedding_matrix = build_embedding_matrix(word_index, emb_glove)
x_train, x_val, y_train, y_val = prepare_model(data_df, tokenizer)
model = build_train_model(word_index, embedding_matrix, x_train, x_val, y_train, y_val)

 46%|████▋     | 34992/75282 [00:00<00:00, 349911.99it/s]

Found 75282 unique tokens.
build embedding matrix


100%|██████████| 75282/75282 [00:00<00:00, 368201.05it/s]


build model
label encoding
Shape of data tensor: (416809, 50)
Shape of label tensor: (416809, 6)
Data train/valid: (333447, 50)/(83362, 50) Label train/valid:  (333447, 6)/(83362, 6)
run model - run and save the model pipeline
Train on 333447 samples, validate on 83362 samples
Epoch 1/30
333447/333447 [==============================] - 257s 772us/step - loss: 0.8855 - acc: 0.6615 - val_loss: 0.7308 - val_acc: 0.6941

Epoch 00001: val_acc improved from -inf to 0.69411, saving model to best_model.b5
Epoch 2/30
333447/333447 [==============================] - 259s 776us/step - loss: 0.5161 - acc: 0.8027 - val_loss: 0.3600 - val_acc: 0.8518

Epoch 00002: val_acc improved from 0.69411 to 0.85176, saving model to best_model.b5
Epoch 3/30
333447/333447 [==============================] - 254s 762us/step - loss: 0.4024 - acc: 0.8426 - val_loss: 0.1714 - val_acc: 0.9170

Epoch 00003: val_acc improved from 0.85176 to 0.91699, saving model to best_model.b5
Epoch 4/30
333447/333447 [===============

In [29]:
validation_model(model, x_val, y_val)

run model - predict validation set
Last validation loss: 1.1920930376163597e-07, accuracy: 0.1360212117433548
Best validation loss: 0.11276268255414988, accuracy: 0.9342146515846252
run model - check prediction accuracy | precision | recall | F1-score
              precision    recall  f1-score   support

           0       0.92      0.96      0.94     11339
           1       0.93      0.86      0.89      9376
           2       0.98      0.92      0.95     28247
           3       0.79      0.93      0.86      6853
           4       0.98      0.96      0.97     24504
           5       0.75      0.96      0.84      3043

    accuracy                           0.93     83362
   macro avg       0.89      0.93      0.91     83362
weighted avg       0.94      0.93      0.94     83362

run model - completed validation
best model saved
